# Castro et al. (2019) Data Processing and Analysis

## Setup

In [1]:
import os
import sys
import glob
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import re
import pickle
from collections import Counter
from functools import reduce

import pandas as pd
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib as mpl

In [3]:
from cvtk.cvtk import TemporalFreqs, TiledTemporalFreqs
from cvtk.cov import stack_temporal_covariances
import cvtk.variant_files as vf
from cvtk.gintervals import GenomicIntervals
from cvtk.pca import FreqPCA
from cvtk.plots import rep_plot_pca, correction_diagnostic_plot
from cvtk.utils import integerize
from cvtk.utils import extract_empirical_nulls_diagonals, extract_temporal_cov_diagonals
from cvtk.cov import stack_replicate_covariances, stack_temporal_covs_by_group
from cvtk.variant_files import VCFFile

In [4]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
mpl.rcParams['figure.figsize'] = (8.0, 4.0)
mpl.rcParams['figure.dpi'] = 200

# Varianta Data Loading

### Load in VCF data

In [5]:
vcf = VCFFile('../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz')

reading file '../data/castro_et_al_2019/beagle_genMap.all.impute.vcf.gz'...


MemoryError: 

Remove fixed sites — those that are not polymorphic in any samples / timepoints. These just needlessly shrink the covariance towards zero.

### Sample Data

The samples names to line/generation mapping was not in a simple text file, but I found the relevant information in the vcftools commands that were at the beginning of the file `Longshanks_F0F17.summary_stats.tar.gz`. From this I created `samples.txt`, which is read in and parsed below.

In [ ]:
samples = pd.read_csv("../data/castro_et_al_2019/samples.txt", header=None, names = ('line', 'individual'))
sample_map = {k:v for k, v in zip(samples['individual'], samples['line'])}

subpop_indices = defaultdict(list)
for i, k in enumerate(vcf.samples):
    subpop_indices[sample_map[k.decode()]].append(i)

From this, we can map the `vcf.geno_mat` table to subpopulation counts. 

In [ ]:
counts_mat = vcf.count_alleles_subpops(subpop_indices)

In [ ]:
vcf.subpops

Now we count the number of diploids in each sample.

In [ ]:
ndiploids = [Counter(sample_map.values())[k] for k in vcf.subpops]

In [ ]:
def parse_samples(x):
    line, gen = x.split('_')
    return (line, gen[1:])

design = [parse_samples(x) for x in vcf.subpops]

In [ ]:
freq_mat_all = vcf.calc_freqs()

In [ ]:
print("number of loci: ", freq_mat_all.shape[1])

With the frequencies calculated, now we filter out all non-segregating sites.

In [ ]:
vcf.remove_fixed()
freq_mat = vcf.calc_freqs()
print("number of loci: ", freq_mat.shape[1])
print("loci not segregating removed: ", freq_mat_all.shape[1] - freq_mat.shape[1])

In [ ]:
gi = vcf.build_gintervals()

## Replicate Covariance Analysis

In [ ]:
tile_width = 10e6
gi.infer_seqlens()
tiles = GenomicIntervals.from_tiles(gi.seqlens, width=tile_width)

In [ ]:
d = TiledTemporalFreqs(tiles, freqs=freq_mat, depths=vcf.N, diploids=ndiploids, samples=design, gintervals=gi)

In [ ]:
d.samples

In [ ]:
autosomes = list(set(gi.intervals.keys()) - set('chrX'))

In [ ]:
covs_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes, average_replicates=False, progress_bar=True)

In [ ]:
covs_cis

In [ ]:
with open('../data/castro_et_al_2019/covs_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_cis)

### Bootstrap the Convergence Correlation

In [ ]:
convergence_corr = d.bootstrap_convergence_corr(B=5000, progress_bar=True)

In [ ]:
convergence_corr

## Analysis Excluding Chromosomes 5 and 10

In [ ]:
autosomes_sans_chr5_and_chr10 = [chr for chr in autosomes if chr not in ('chr5', 'chr10')]

In [ ]:
covs_sans_chr5_and_chr1_cis = d.bootstrap_cov(B=5000, keep_seqids=autosomes_sans_chr5_and_chr10, 
                                              average_replicates=False, progress_bar=True)

In [ ]:
covs_sans_chr5_and_chr1_cis

In [ ]:
with open('../data/castro_et_al_2019/covs_sans_chr5_and_chr1_bootstrap_10e6.npy', 'wb') as f:
    np.save(f, covs_sans_chr5_and_chr1_cis)